# ANALYSIS OF MTOR

This notebook contains an analysis of the mTOR system described in Varusai and Nguyen, 2018.

We consider a model of the signaling network for mTOR, the mechanistic Target of Rapamycin (mTOR), an evolutionarily conserved network that controls key cellular processes, especially cell growth and metabolism. The signalling cascade is activated by a variety of receptor tyrosine kinases, most notably the Insulin Receptor.

<img src="images/mTOR-System.png"
     alt="Markdown Monster icon" 
     width="500" height="600"
     style="float: left; margin-right: 10px;" />

In [ ]:
import controlSBML as ctl

import control
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
ctl.__version__

# Problem Statement

The biological problem we address is controlling the concentration of phosphorylated mTORC1 through phosphorylated insulin. \ccite{varusai_dynamic_2018} describes this in terms of the species pIRS. The system we study has the single input $pIRS$ and the output mTORC1. It turns out that the mTOR system is bistable with respect to the insulin. Thus, our control objective is to regulate the outputs in the high insulin region where V1 $> 4$.
* Outputs: pmTORC1, pmTORC2
* Inputs: V1
* Control problem: regulate pmTORC1, pmTORC2

## mTOR Model

In [ ]:
URL = "https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000823.2?filename=Varusai2018.xml"
INPUT_NAME = "IR"
OUTPUT_NAME = "pmTORC1"
SYSTEM = ctl.SBMLSystem(URL, input_names=[INPUT_NAME], output_names=[OUTPUT_NAME],
                       is_fixed_input_species=True)

In [ ]:
ts = SYSTEM.plotModel(figsize=(5,5), end_time=2000, markers=np.repeat("", 20))
for column in ts.columns:
    if not column in [INPUT_NAME, OUTPUT_NAME]:
        del ts[column]
times = ts.index/1000
plt.plot(times, ts[OUTPUT_NAME], color="green")
ax = plt.gca()
ax2 = ax.twinx()
ax2.plot(times, ts[INPUT_NAME], color="red")
ax.set_ylabel(OUTPUT_NAME, color="green")
ax2.set_ylabel(INPUT_NAME, color="red")
ax.set_xlabel("time")

In [ ]:
print(SYSTEM.getValidInputs())

In [ ]:
STAIRCASE = ctl.Staircase(initial_value=5, final_value=10, num_step=5)
BUILDER = ctl.SISOTransferFunctionBuilder(SYSTEM)
response_ts = BUILDER.makeStaircaseResponse(staircase=STAIRCASE, end_time=5000, points_per_time=10)
BUILDER.plotStaircaseResponse(response_ts, figsize=(5,5))

* The system is controllable in the region V1 $>4$, although the control decreases with larger V1 for both complexes.
* The operating region for V1 implies an operating region for pmTORC1 of 180 to 220 and for pmTORC2 0.7 to 1.0.

# System Identification

In [ ]:
fitter_result = BUILDER.fitTransferFunction(num_numerator=2, num_denominator=3, staircase=STAIRCASE, 
                                            fit_start_time=0, start_time=0, end_time=5000)
BUILDER.plotFitTransferFunction(fitter_result, figsize=(5,5))

# Testbed Construction

Here, we build a closed loop system where the open loop is the SBML model of the simulated system. Our has a PI controller.

In [ ]:
setpoint = 0.2
ts, builder = SYSTEM.simulateSISOClosedLoop(kp=6.8,  end_time=10000, is_steady_state=False, 
                                            setpoint=setpoint, num_point=1000, sign=1,
                                            )
SYSTEM.plotSISOClosedLoop(ts, setpoint, markers=["", ""], figsize=(5,5))

We are able to simulate the closed loop system, but it's not easy to find design parameters such that: (a) settling times are short; (b) there is not overshoot; and (c) all concentrations are non-negative.

**Notes**
1. Need to use positive feedback. Not clear from the controlability curves.
2. Introducing even a little I behavior makes the controller worse.
3. There is a multimodal behavior of the network that radically changes the long term steady state values. Could this be analyzed by decomposing the system to have multiple tandem controllers. The first controls the effect of IR on pIRS; the second controls the effect of pIRS on pmTORC1.

In [ ]:
builder

# Design and Evaluation